In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_pickle("benchmark_linear_bootstrap.pkl")
df1["delta_0"] = df1["params_true_0"] - df1["params_0"]
df1["delta_1"] = df1["params_true_1"] - df1["params_1"]
df1["pull_0"] = df1["delta_0"] / df1["errors_0"]
df1["pull_1"] = df1["delta_1"] / df1["errors_1"]

In [ ]:
df1.head(10)

In [ ]:
df2 = pd.read_pickle("benchmark_linear_eachfit.pkl")
df2.head(5)

In [ ]:
df1_tf = df1.query("fitter_name=='Tensorflow_BFGS'")
df1_scipy= df1.query("fitter_name=='Scipy_LM'")
df1_torch= df1.query("fitter_name=='TPytorch_LBFGS'")
N = len(df1_tf.index)

In [ ]:
df1_tf.head(5)

In [ ]:
df1_scipy.head(5)

In [ ]:
df1_torch.head(5)

In [ ]:
print("Tensorflow BFGS fitter: ", df1_tf["delta_0"].mean(), "+-", df1_tf["errors_0"].mean()/np.sqrt(N) )
print("Scipy LM fitter: ", df1_scipy["delta_0"].mean(), "+-", df1_scipy["errors_0"].mean()/np.sqrt(N))
print("Pytorch LBFGS fitter: ", df1_torch["delta_0"].mean(), "+-", df1_torch["errors_0"].mean()/np.sqrt(N))

In [ ]:
print(df1_tf["delta_0"].std(),df1_tf["bs_std_0"].mean(), df1_tf["errors_0"].mean())
print(df1_scipy["delta_0"].std(),df1_scipy["bs_std_0"].mean(), df1_scipy["errors_0"].mean())
print(df1_torch["delta_0"].std(),df1_torch["bs_std_0"].mean(), df1_torch["errors_0"].mean())

In [ ]:
df1_tf["delta_0"].hist()
df1_scipy["delta_0"].hist()
df1_torch["delta_0"].hist()

In [ ]:
df1_torch["pull_0"].hist()

This seems suspicious, pytorch fitter seems to overestimate the errors

In [ ]:
df1_scipy["pull_0"].hist()